# 제3 고지 : 고차 미분 계산
## STEP 36 : 고차 미분 이외의 용도

지금까지 여러 단계를 거쳐 DeZero를 구현하여 **고차미분을 수행하기 위해 역전파의 계산그래프 '연결'**를 만들었다.  
이를 활용하여 고차 미분 외에도 어떻게 활용될 수 있는지 살펴본다.


### 36.1 double backprop의 용도 

새로운 DeZero는 **역전파로 수행한 계산에 대해 또 다시 역전파 할 수 있는 `double backpropagation`** 를 지원한다. 
 
우선 다음 문제를 생각해보자.
**문제**: 다음의 두 식이 주어졌을 때 $x=2.0$에서 x에 대한 z의 미분 $\frac{\partial z}{\partial x}$ 를 구하라.
<p align='center'>
    <img src='../assets/%EC%8B%9D%2036.1.png' align='center' width='50%'>
    <img src='../assets/%EC%8B%9D%2036.2.png' align='center' width='50%'>
</p>

지금까지와 같은 미분 계산이지만,[식36.2] 에 이미 미분이 포함되어 있는 점이 다르다. 즉, **미분이 포함된 식에서 다시 한번 미분**해야 한다.  
구현에 앞서 다음 수식 전개 과정을 살펴보자. 
$$
\begin{aligned}
\frac{\partial y }{\partial x} &= 2x \\ 
z &= (\frac{\partial y}{\partial x})^3 + y = 8x^3+x^2 \\
\Rightarrow \frac{\partial z}{\partial x} &= 24x^2 + 2x 
\end{aligned}
$$

주의할것은 위 식에서 $\frac{\partial y}{\partial x}$는 값이 아니라 $x$의 식이다. 만약 여기서 $x=2.0$ 일때 $\frac{\partial y}{\partial x}$ 의 값을 찾고, 그 값을 $z = (\frac{\partial y}{\partial x})^3 + y $ 에 대입하면 올바른 결과를 얻을 수 없다.




In [4]:
import sys

sys.path.append("..")

import numpy as np
from dezero import Variable

x = Variable(np.array(2.0))
y = x ** 2 

y.backward(create_graph=True)
gx = x.grad 
print(f"dy/dx : {x.grad}")

x.cleargrad() 

z = gx ** 3 + y 
z.backward()
print(f"dz/dx : {x.grad}")


dy/dx : variable(4.0)
dz/dx : variable(100.0)


### 36.2 딥러닝 연구에서의 사용 예

<p align='center'>
    <img src='../assets/%EA%B7%B8%EB%A6%BC%2036-1.png' align='center' width='50%'>
</p>

딥러닝에서 `double backprop` 을 사용하는 연구는 여러가지 이지만, 위의 [수식](https://arxiv.org/pdf/1704.00028.pdf)과 같이 최적화 하는 식에 gradient가 들어 있다는 것이다.  
즉, 
1. 첫번째 역전파에서 gradient를 구한 후 해당 gradient를 사용하여 $\mathcal{L}$ 을 계산한다.
2. $\mathcal{L}$을 최적화 하기 위해 두 번째 역전파를 수행한다.
